## CHEME 5660: Binomial Price Estimate of a Single Risky Asset

<img src="./figs/Fig-Binomial-Lattice-Schematic.png" style="margin:auto; width:30%"/>

### Binomial lattice model
A binomial lattice model assumes that each discrete time increment, the state of the system, e.g., the share price of equity, the spot rate, etc., can either increase by a factor $u$ with probability $p$ or decrease by a factor $d$ with probability $(1-p)$. Different models can be developed for the specific values of the tuple $(u,d,p)$. One particular model is the Cox, Ross, and Rubinstein (CRR) model:

* [Cox, J. C.; Ross, S. A.; Rubinstein, M. (1979). "Option pricing: A simplified approach". Journal of Financial Economics. 7 (3): 229. CiteSeerX 10.1.1.379.7582. doi:10.1016/0304-405X(79)90015-1](https://www.sciencedirect.com/science/article/pii/0304405X79900151?via%3Dihub)

#### Cox, Ross and Rubinstein (CRR) model
The [CRR binomial lattice model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) was initially developed for options pricing in 1979. However, one of the critical aspects of estimating an option’s price is calculating the underlying asset’s share price. Thus, let's use the [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) to compute the share price of a stock, Advanced Micro Devices, Inc, with the ticker symbol [AMD](https://finance.yahoo.com/quote/AMD?.tsrc=applewf). In the [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) model, the `up` and `down` moves are symmetric:

$$ud = 1$$

where the magnitude of an `up` move $u$ is given by:

$$u = \exp(\sigma\sqrt{\Delta{T}})$$

The quantity $\sigma$ denotes a _volatility parameter_, and $\Delta{T}$ represents the time step. The probability $p$ of an `up` move in a [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) is given by:

$$p = \frac{\exp(\mu\sqrt{\Delta{T}}) - d}{u - d}$$

where $\mu$ denotes a _return parameter_. In the [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) model paradigm, the return parameter $\mu$ and the volatility parameter $\sigma$ take on common values:
* The return parameter $\mu$ is a _risk-free_ rate of return; the _risk-free_ rate $\bar{r}$ can be approximated by the [yield on T = 10-year United States Treasury debt security](https://ycharts.com/indicators/10_year_treasury_rate). 
* The volatility parameter $\sigma$ is the [implied volatility](https://www.investopedia.com/terms/i/iv.asp); the implied volatility is the market's view of the likelihood of changes in a given security's price.

### Lab setup
The code block below installs (and loads) any [Julia](https://julialang.org) packages that we need to complete the calculations. 

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-2-Binomial-Pricing-Single-Assets`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-2-Binomial-Pricing-Single-Assets/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-2-Binomial-Pricing-Single-Assets/Manifest.toml`


In [2]:
using PQEcolaPoint
using DataFrames
using CSV
using Statistics

### Load the lab 2 code library
The call to the `include` function loads the `CHEME-5660-Lab-2-Library.jl` library into the notebook; the library contains types and functions we use during the lab:

* The `E(X::Array{Float64,1},p::Array{Float64,1}) -> Float64` and `Var(X::Array{Float64,1}, p::Array{Float64,1}) -> Float64` functions compute the expectation and variance of the binomial price estimates given an array `X::Array{Float64,1}` of price values and associated probabilities `p::Array{Float64,1}`.
* The `build_probability_dictionary(model::CRRLatticeModel, levels::Int64) -> Dict{Int64, Array{Float64,1}}` function constructs a dictionary of probabilities for each level of the tree; keys are tree levels.
* The `build_nodes_dictionary(levels::Int64) -> Dict{Int64,Array{Int64,1}}` function constructs a dictionary of node indexs for each level of the tree; keys are the tree levels.

In [3]:
# load the code library -
include("CHEME-5660-Lab-2-Library.jl");

#### a) Load experimental data for AMD
Load the historical OHLC data set for Advanced Micro Devices, Inc with ticker symbol [AMD](https://finance.yahoo.com/quote/AMD?.tsrc=applewf) into a [DataFrame](https://dataframes.juliadata.org/stable/). The OHLC data is stored in a comma seperated value (CSV) file format; use the [CSV](https://csv.juliadata.org/stable/) package to read the data and load into a [DataFrame](https://dataframes.juliadata.org/stable/).

In [4]:
df = CSV.read("./data/AMD-OHLC-2020-8-25-to-2022-09-27.csv", DataFrame);

#### b) Estimate CRR model parameters for AMD
In the [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) model paradigm, the return parameter $\mu$ and the volatility parameter $\sigma$ take on common values:
* The return parameter $\mu$ is a _risk-free_ rate of return; the _risk-free_ rate $\bar{r}$ can be approximated by the [yield on T = 10-year United States Treasury debt security](https://ycharts.com/indicators/10_year_treasury_rate). 
* The volatility parameter $\sigma$ is related to the [implied volatility](https://www.investopedia.com/terms/i/iv.asp); the implied volatility is the market's view of the likelihood of changes in a given security's price in the next year.

In [5]:
# How many days do we want to simulation?
L = 30; # units:days number of tree levels (note: the tree data model is 1 based)

In [6]:
# Set the risk free rate -
r̄ₘ = 0.04;

In [7]:
# What is the Implied Volatility (IV) -
IV = 58.3 # implied volatility 
σₘ = (IV/100.0)*sqrt(L/365)

0.16714090520211028

In [8]:
# What is the initial share price?
Sₒ = df[end-5,:close]

75.25

In [9]:
# build a CRR lattice model -
model = build(CRRLatticeModel; number_of_levels=(L+1), Sₒ = Sₒ, σ = σₘ, μ = r̄ₘ, T = (L/365.0));

In [10]:
# Get the estimated prices at all the nodes -
P = model.data[:,1];

In [11]:
# What else is stored in the model?
p = model.p
u = model.u
d = model.d
println("The probability of an UP move: $(p) where the UP factor u = $(u)")

The probability of an UP move: 0.5040101077233302 where the UP factor u = 1.0086434330234348


#### c) Analysis of the binomial lattice price values

#### Build: node dictionary `id`

In [12]:
id = build_nodes_dictionary(L); # zero based

#### Build: probability dictionary `pd`

In [13]:
pd = build_probability_dictionary(model, L); # zero based

#### Extract: prices at $T=\star$

In [14]:
# get the prices and probability for some T
T = 5
p = pd[T];
X = P[id[T]]

6-element Vector{Float64}:
 78.55879817247519
 77.21816913904887
 75.90041833501347
 74.60515533664474
 73.33199638291434
 72.08056426178885

#### Compute the expectation and the variance of the estimated price values
The expectation for a discrete random variable $X$, denoted by $\mathbb{E}(X)$ is defined as:

$$\mathbb{E}(X) = \sum_{x\in{X\left(\Omega\right)}}xp_{X}(x)$$

while the variance of $X$, denoted by $\text{Var}(X)$ is defined as:

$$\text{Var}(X) = \mathbb{E}(X^2) - \mathbb{E}(X)^2$$

In [15]:
# Compute the expected value
S̄ = E(X,p)

75.2899133654011

In [16]:
# Compute the variance and stdev
σ̄ = sqrt(Var(X,p))

1.4488231990252862

In [17]:
println("In T = $(T) days we expect the share price of AMD to be $(S̄) ± $(σ̄) USD/share")

In T = 5 days we expect the share price of AMD to be 75.2899133654011 ± 1.4488231990252862 USD/share
